In [7]:
import numpy as np

In [34]:
class activation_fn ():
    """ define activation functions that will be needed in the DNN"""
    
    # sigmoid function for forward and backward pass
    def sigmoid(self, Z=None, A=None, derivative=False):
        if derivative:
            return A * (1 - A)
        else:
            return 1 / (1 + np.exp(-Z))
        

In [35]:
test = activation_fn()
test.sigmoid(Z=3)

0.9525741268224334

In [36]:
test.sigmoid(A=3, derivative=True)

-6